### Scrape: http://www.usachurches.org/search/searchbystate.php?start=20&query=&denomination=&keywords=&city=&size=&statea=nj&worshipstyle=

In [1]:
url = "http://www.usachurches.org/search/searchbystate.php?start=20&query=&denomination=&keywords=&city=&size=&statea=nj&worshipstyle="

In [6]:
import os, logging, sys, argparse, email, base64, time, pytz, html5lib, requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from itertools import chain

In [4]:
from IPython.display import IFrame
IFrame(url, width=800, height=650)

## Process
### Step 1:
- for each of the 11 pages get all links for each church
    - val = 20 * page num (1 - 11)
    -  url form: f"http://www.usachurches.org/search/searchbystate.php?start={val}&query=&denomination=&keywords=&city=&size=&statea=nj&worshipstyle="
### Step 2:
- load each church url and get data:
    - church name
    - denomination
    - address
    - leaders
    - phone number
    - location (county)
    - website
---    

## Step 1

In [29]:
def get_step1_links(res_index):
    val = 20 * res_index
    url =  f"http://www.usachurches.org/search/searchbystate.php?start={val}&query=&denomination=&keywords=&city=&size=&statea=nj&worshipstyle="
    print (url)
    soup = BeautifulSoup(requests.get(url).content,'html5lib')
    links = list({link['href'] for link in soup.findAll('div',{'col span_6_of_8 content'})[0].findAll('a') if link['href'].startswith("http://www.usachurches.org/church/")})
    return links

In [37]:
urls = list(chain.from_iterable([get_step1_links(i) for i in range(11)]))

http://www.usachurches.org/search/searchbystate.php?start=0&query=&denomination=&keywords=&city=&size=&statea=nj&worshipstyle=
http://www.usachurches.org/search/searchbystate.php?start=20&query=&denomination=&keywords=&city=&size=&statea=nj&worshipstyle=
http://www.usachurches.org/search/searchbystate.php?start=40&query=&denomination=&keywords=&city=&size=&statea=nj&worshipstyle=
http://www.usachurches.org/search/searchbystate.php?start=60&query=&denomination=&keywords=&city=&size=&statea=nj&worshipstyle=
http://www.usachurches.org/search/searchbystate.php?start=80&query=&denomination=&keywords=&city=&size=&statea=nj&worshipstyle=
http://www.usachurches.org/search/searchbystate.php?start=100&query=&denomination=&keywords=&city=&size=&statea=nj&worshipstyle=
http://www.usachurches.org/search/searchbystate.php?start=120&query=&denomination=&keywords=&city=&size=&statea=nj&worshipstyle=
http://www.usachurches.org/search/searchbystate.php?start=140&query=&denomination=&keywords=&city=&size

In [38]:
urls

['http://www.usachurches.org/church/calvary-chapel-of-monmouth-county.htm',
 'http://www.usachurches.org/church/parkunitedmethodistchurch.htm',
 'http://www.usachurches.org/church/grace-christian-fellowship-of-south-jersey.htm',
 'http://www.usachurches.org/church/warren-point-presbyterian-church.htm',
 'http://www.usachurches.org/church/hope-center-tabernacle.htm',
 'http://www.usachurches.org/church/the-old-bridge-baptist-church.htm',
 'http://www.usachurches.org/church/full-gospel-church.htm',
 'http://www.usachurches.org/church/paterson-assembly-of-god.htm',
 'http://www.usachurches.org/church/notre-dame-church.htm',
 'http://www.usachurches.org/church/st-pauls-united-methodist-church-port-republic.htm',
 'http://www.usachurches.org/church/east-dover-baptist-church.htm',
 'http://www.usachurches.org/church/new-vision-full-gospel-baptist-church.htm',
 'http://www.usachurches.org/church/boonton-united-methodist-church.htm',
 'http://www.usachurches.org/church/circle-of-hope.htm',
 'h

# Step 2
- grab data for each desired field for each url
- combine results into DataFrame
- save output to csv file

In [40]:
name_ = []
sect_ = []
address_ = []
csz_ = []
county_ = []
leader_ = []
phone_ = []

In [42]:
soup = BeautifulSoup(requests.get('http://www.usachurches.org/church/the-church-of-jesus-christ.htm').content,'html5lib')

In [44]:
def get_soup(url):
    return BeautifulSoup(requests.get(url).content,'html5lib')

In [132]:
df = pd.DataFrame({
    "Church Name":name_,
    "Leaders":leader_,
    "Sect":sect_,
    "Address":address_,
    "City State Zip":csz_,
    "County":county_,
    "Phone":phone_
})

In [131]:
name_ = []
sect_ = []
address_ = []
csz_ = []
county_ = []
leader_ = []
phone_ = []

for _,url in enumerate(urls):
    print (f"{_}: Acquiring data for {url}\n")
    soup = get_soup(url)
    
    name = soup.h1.contents[0]
    try:
        sect = soup.findAll('div',{'class':'col span_5_of_8'})[0].p.a.contents[0]
    except:
        sect = soup.findAll('div',{'class':'col span_5_of_8'})[0].p.contents[0]
    leader = '\n'.join(soup.findAll('div',{'class':'col span_1_of_2'})[0].span.contents[::2])
    address = soup.findAll('div',{'class':'col span_5_of_8'})[0].p.contents[2].strip()
    csz = soup.findAll('div',{'class':'col span_5_of_8'})[0].p.contents[4].strip()
    county = soup.findAll('div',{'class':'leftsetp'})[1].li.span.contents[0].replace('Location: ','')
    try:
        phone = soup.findAll('ul',{'class':'fa-ul formore'})[0].li.contents[1]
    except:
        phone = ''
    name_.append(name)
    sect_.append(sect)
    leader_.append(leader)
    address_.append(address)
    csz_.append(csz)
    county_.append(county)
    phone_.append(phone)

0: Acquiring data for http://www.usachurches.org/church/calvary-chapel-of-monmouth-county.htm

1: Acquiring data for http://www.usachurches.org/church/parkunitedmethodistchurch.htm

2: Acquiring data for http://www.usachurches.org/church/grace-christian-fellowship-of-south-jersey.htm

3: Acquiring data for http://www.usachurches.org/church/warren-point-presbyterian-church.htm

4: Acquiring data for http://www.usachurches.org/church/hope-center-tabernacle.htm

5: Acquiring data for http://www.usachurches.org/church/the-old-bridge-baptist-church.htm

6: Acquiring data for http://www.usachurches.org/church/full-gospel-church.htm

7: Acquiring data for http://www.usachurches.org/church/paterson-assembly-of-god.htm

8: Acquiring data for http://www.usachurches.org/church/notre-dame-church.htm

9: Acquiring data for http://www.usachurches.org/church/st-pauls-united-methodist-church-port-republic.htm

10: Acquiring data for http://www.usachurches.org/church/east-dover-baptist-church.htm

11: 

In [133]:
df

,Church Name,Leaders,Sect,Address,City State Zip,County,Phone
0,Calvary Chapel of Monmouth County,"Rev. Dr. Richard L. Shaw PhD, Pastor",Non-Denominational / Independent,810 New Jersey 36,"Leonardo, NJ 07737",Monmouth County,(732) 497-2444
1,Park United Methodist Church,"Esther Morales, Pastor",United Methodist Church,51 Clifton Terrace,"Weehawken, NJ 07086",Hudson County,(201) 867-9161
2,Grace Christian Fellowship of South Jersey,"Rev. Robert C. Brayerton, Sr., Senior Pastor\n...",Non-Denominational / Independent,632 South Pennsville Auburn Road,"Carneys Point, NJ 08069",Salem County,(856) 376-3627
3,Warren Point Presbyterian Church,"Rev. Jeffrey N. Leininger, Pastor",Presbyterian Church U.S.A.,Broadway & 17th Street,"Fair Lawn, NJ 07410",Bergen County,(201) 796-3662
4,Hope Center Tabernacle,"Rev. Mario Gonzalez, Senior Pastor\nRev. Leigh...",Assemblies of God,110-116 Cambridge Avenue,"Jersey City, NJ 07307",Hudson County,(201) 798-1234
5,The Old Bridge Baptist Church,"Stan Mistak, Pastor",Independent Baptist,21 Kossman Street,"East Brunswick, NJ 08816",Middlesex County,(732) 257-7959
6,Full Gospel Church,"Rev. Edward Spinola, Interim",Assemblies of God,2649 East Hurley Pond Road,"Wall Township, NJ 07719",Monmouth County,(732) 681-5335
7,Paterson Assembly of God,"Rev. Dr. Bobby Moore, Pastor\nJanice Moore, Fi...",Assemblies of God,385 Park Avenue,"Paterson, NJ 07504",Passaic County,(973) 279-0012
8,Notre Dame Church,"Fr. Anthony Randazzo, Pastor",Roman Catholic Church,359 Central Avenue,"North Caldwell, NJ 07006",Essex County,(973) 226-0979
9,St. Paul's United Methodist Church,"Frank P. Brown, Pastor",United Methodist Church,118 Main Street,"Port Republic, NJ 08241",Atlantic County,(609) 652-6946


In [138]:
df.to_csv('NJ_Church_list.csv',encoding='utf-8',index=False)